<a href="https://colab.research.google.com/github/anl078/ECE204Repo/blob/Solana/R_dNdScv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if (!requireNamespace("BiocManager", quietly = TRUE)) {
  install.packages("BiocManager")
}

# Install required dependencies using Bioconductor
BiocManager::install(c("Rcpp", "dplyr", "ggplot2", "data.table", "magrittr"))

library(devtools); install_github("im3sanger/dndscv")
library(dndscv)
library(dplyr)
library(magrittr)
library(ggplot2)
library(Rcpp)
library(data.table)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.21 (BiocManager 1.30.25), R 4.5.0 (2025-04-11)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'Rcpp' 'dplyr' 'ggplot2' 'data.table'
  'magrittr'”
Installing package(s) 'BiocVersion'

Old packages: 'systemfonts', 'textshaping', 'utf8'

Loading required package: usethis




formatR      (NA -> 1.14    ) [CRAN]
futile.op... (NA -> 1.0.1   ) [CRAN]
lambda.r     (NA -> 1.2.4   ) [CRAN]
BH           (NA -> 1.87.0-1) [CRAN]
snow         (NA -> 0.4-4   ) [CRAN]
futile.lo... (NA -> 1.4.3   ) [CRAN]
IRanges      (NA -> 2.42.0  ) [CRAN]
S4Vectors    (NA -> 0.46.0  ) [CRAN]
BiocGenerics (NA -> 0.54.0  ) [CRAN]
GenomeInf... (NA -> 1.2.14  ) [CRAN]
UCSC.utils   (NA -> 1.4.0   ) [CRAN]
GenomeInfoDb (NA -> 1.44.0  ) [CRAN]
XVector      (NA -> 0.48.0  ) [CRAN]
RcppArmad... (NA -> 14.4.2-1) [CRAN]
sp           (NA -> 2.2-0   ) [CRAN]
pixmap       (NA -> 0.4-13  ) [CRAN]
Rhtslib      (NA -> 3.4.0   ) [CRAN]
BiocParallel (NA -> 1.42.0  ) [CRAN]
bitops       (NA -> 1.0-9   ) [CRAN]
Biostrings   (NA -> 2.76.0  ) [CRAN]
GenomicRa... (NA -> 1.60.0  ) [CRAN]
segmented    (NA -> 2.1-4   ) [CRAN]
ade4         (NA -> 1.7-23  ) [CRAN]
plyr         (NA -> 1.8.9   ) [CRAN]
poilog       (NA -> 0.4.2.1 ) [CRAN]
Rsamtools    (NA -> 2.24.0  ) [CRAN]
seqinr       (NA -> 4.2-36  ) [CRAN]


Installing 27 packages: formatR, futile.options, lambda.r, BH, snow, futile.logger, IRanges, S4Vectors, BiocGenerics, GenomeInfoDbData, UCSC.utils, GenomeInfoDb, XVector, RcppArmadillo, sp, pixmap, Rhtslib, BiocParallel, bitops, Biostrings, GenomicRanges, segmented, ade4, plyr, poilog, Rsamtools, seqinr

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpiZ7Ge9/remotes2a2abe815c/im3sanger-dndscv-dab351a/DESCRIPTION’ ... OK
* preparing ‘dndscv’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* looking to see if a ‘data/datalist’ file should be added
  NB: this package now depends on R (>= 3.5.0)
  serialize/load version 3 cannot be read in older versions of R.
  File(s) containing such objects:
    ‘dndscv/data/covariates_hg19_chrx.rda’
* building ‘dndscv_0.0.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“replacing previous import ‘Biostrings::translate’ by ‘seqinr::translate’ when loading ‘dndscv’”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last




In [3]:
### READ MUTATION DATA ###
df <- read.table("TCGA.STAD.mutations.txt", header=TRUE, sep="\t")

### PREPROCESSING ###

## Remove hypermutators (top 1% by mutation count)
mutation_counts <- df %>%
  count(patient_id, name = "mut_count")

cutoff <- quantile(mutation_counts$mut_count, 0.99)

reg_mutaters <- df %>%
  inner_join(mutation_counts, by = "patient_id") %>%
  filter(mut_count <= cutoff) %>%
  select(-mut_count)

# Remove genes with less than 10 mutations
filtered_mutations <- reg_mutaters %>%
  filter(!is.na(Hugo_Symbol)) %>%
  group_by(Hugo_Symbol) %>%
  filter(n() >= 10) %>%
  ungroup()

# Only keep SNP
filtered_mutations <- filtered_mutations %>%
  filter(VARIANT_CLASS == "SNV")

## Retain pertinent columns
filtered_mutations <- filtered_mutations[c("patient_id","Hugo_Symbol","Chromosome", "Start_Position","Reference_Allele", "Tumor_Seq_Allele1","Tumor_Seq_Allele2")]

# Clean up: only keep the allele with the mutatation
dndscv_data <- filtered_mutations %>%
  mutate(
    mut = case_when(
      Tumor_Seq_Allele1 != Reference_Allele & !is.na(Tumor_Seq_Allele1) ~ Tumor_Seq_Allele1,
      Tumor_Seq_Allele2 != Reference_Allele & !is.na(Tumor_Seq_Allele2) ~ Tumor_Seq_Allele2,
      TRUE ~ NA_character_
    ),
    chr = as.character(Chromosome),  # Ensure chromosome is a character type
    pos = Start_Position,
    ref = Reference_Allele,
  ) %>%
  select(sampleID = patient_id, chr, pos, ref, mut)

write.table(dndscv_data, file = "dndscv_data.tsv", sep = "\t", row.names = FALSE, quote = FALSE) # save input df

Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“number of items read is not a multiple of the number of columns”


In [4]:
### RUN dNdScv
data("dataset_simbreast", package="dndscv")
dndsout = dndscv(dndscv_data)

[1] Loading the environment...

[2] Annotating the mutations...

Warning message in dndscv(dndscv_data):
“Same mutations observed in different sampleIDs. Please verify that these are independent events and remove duplicates otherwise.”
    Note: 22 mutations removed for exceeding the limit of mutations per gene per sample (see the max_muts_per_gene_per_sample argument in dndscv)

[3] Estimating global rates...

[4] Running dNdSloc...

[5] Running dNdScv...

Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs produced”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs produced”
    Regression model for substitutions (theta = 8.14e+03).



In [5]:
sel_cv = dndsout$sel_cv
print(head(sel_cv), digits = 2)

      gene_name n_syn n_mis n_non n_spl wmis_cv wnon_cv wspl_cv pmis_cv
18057      TP53     0     7     3     1    1561    9494    9494       0
9768      LRP1B     4    10     1     1     237     407     407       0
6584        FLG     1    12     1     0     295     277     277       0
18453       TTN     8    24     0     0      71       0       0       0
10901     MUC16     5    17     0     0     139       0       0       0
6346       FAT4     4    13     0     0     265       0       0       0
      ptrunc_cv pallsubs_cv qmis_cv qtrunc_cv qallsubs_cv
18057   6.7e-16           0       0   1.3e-11           0
9768    7.6e-06           0       0   4.4e-02           0
6584    2.4e-03           0       0   1.0e+00           0
18453   8.0e-01           0       0   1.0e+00           0
10901   9.1e-01           0       0   1.0e+00           0
6346    9.4e-01           0       0   1.0e+00           0


Note that no gene has enough truncating and splice site mutations to fit the 3-parameter model that generates qglobal_cv (due to input of SNV only). We use qallsubs_cv (FDR-adjusted p-value for all nonsynonymous mutations) to rank genes.

In [15]:
results <- top_n(sel_cv,-30,qallsubs_cv)
# top_drivers <- top_n(results,10,pglobal_cv) # breaking tie with p-value if ties with q
# top_drivers <- top_n(top_drivers,10,n_ind) # breaking tie with number of

results$percentage_samples <- sapply(
  results$gene_name,
  function(gene) {
    length(unique(filtered_mutations$patient_id[filtered_mutations$Hugo_Symbol == gene])) / # number of mutations for a gene
      length(unique(filtered_mutations$patient_id)) # total number mutations (after filtering ofc)
  }
)

write.table(results, file = "dndscv_package_drivers.tsv", sep = "\t", row.names = FALSE, quote = FALSE)

results

gene_name,n_syn,n_mis,n_non,n_spl,wmis_cv,wnon_cv,wspl_cv,pmis_cv,ptrunc_cv,pallsubs_cv,qmis_cv,qtrunc_cv,qallsubs_cv,percentage_samples
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TP53,0,7,3,1,1560.63841,9493.6594,9493.6594,0.000000e+00,6.661338e-16,0.000000e+00,0.000000e+00,1.338329e-11,0.000000e+00,0.2631579
LRP1B,4,10,1,1,236.83257,407.3807,407.3807,0.000000e+00,7.550975e-06,0.000000e+00,0.000000e+00,4.403099e-02,0.000000e+00,0.2105263
FLG,1,12,1,0,294.68043,276.6120,276.6120,0.000000e+00,2.351947e-03,0.000000e+00,0.000000e+00,9.973377e-01,0.000000e+00,0.2368421
TTN,8,24,0,0,71.33003,0.0000,0.0000,0.000000e+00,8.004183e-01,0.000000e+00,0.000000e+00,9.973377e-01,0.000000e+00,0.3947368
MUC16,5,17,0,0,139.00742,0.0000,0.0000,0.000000e+00,9.114424e-01,0.000000e+00,0.000000e+00,9.973377e-01,0.000000e+00,0.4210526
FAT4,4,13,0,0,264.77080,0.0000,0.0000,0.000000e+00,9.408534e-01,0.000000e+00,0.000000e+00,9.973377e-01,0.000000e+00,0.3157895
RNF213,1,8,0,1,135.33323,232.8934,232.8934,2.442491e-15,2.836711e-03,3.330669e-16,4.907208e-12,9.973377e-01,9.559496e-13,0.1842105
DNAH9,1,8,0,0,173.34333,0.0000,0.0000,3.330669e-16,9.266185e-01,3.441691e-15,8.364559e-13,9.973377e-01,8.426519e-12,0.2105263
FAT3,1,8,0,0,171.55724,0.0000,0.0000,3.330669e-16,9.411749e-01,3.774758e-15,8.364559e-13,9.973377e-01,8.426519e-12,0.2105263


In [18]:
signif_genes = results[results$qallsubs_cv<0.1, c("gene_name","qallsubs_cv","percentage_samples")]
rownames(signif_genes) = NULL
signif_genes

gene_name,qallsubs_cv,percentage_samples
<chr>,<dbl>,<dbl>
TP53,0.000000e+00,0.2631579
LRP1B,0.000000e+00,0.2105263
FLG,0.000000e+00,0.2368421
TTN,0.000000e+00,0.3947368
MUC16,0.000000e+00,0.4210526
FAT4,0.000000e+00,0.3157895
RNF213,9.559496e-13,0.1842105
DNAH9,8.426519e-12,0.2105263
FAT3,8.426519e-12,0.2105263


(dndsout$globaldnds) table with the global MLEs for the dN/dS ratios across all genes. dN/dS ratios with associated confidence intervals are calculated for missense, nonsense and essential splice site substitutions separately, as well as for all non-synonymous substitutions (wall) and for all truncating substitutions together (wtru), which include nonsense and essential splice site mutations.


In [11]:
print(dndsout$globaldnds)

     name       mle     cilow   cihigh
wmis wmis 1.3789981 1.0105016 1.881873
wnon wnon 1.7888116 0.9160508 3.493089
wspl wspl 0.9122673 0.3117680 2.669394
wtru wtru 1.4508945 0.8028623 2.621988
wall wall 1.3917662 1.0234687 1.892596


annotated table of coding mutations (annotmuts)

In [12]:
head(dndsout$annotmuts)

sampleID,chr,pos,ref,mut,gene,strand,ref_cod,mut_cod,ref3_cod,mut3_cod,aachange,ntchange,codonsub,impact,pid
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr[1d]>,<chr[1d]>,<chr[1d]>,<chr[1d]>,<chr[1d]>,<chr[1d]>,<chr>
TCGA-3M-AB46,2,140997082,C,G,LRP1B,-1,G,C,TGC,TCC,V4448V,G13344C,GTG>GTC,Synonymous,ENSP00000374135
TCGA-3M-AB46,2,141356383,C,A,LRP1B,-1,G,T,GGA,GTA,W2337C,G7011T,TGG>TGT,Missense,ENSP00000374135
TCGA-3M-AB46,2,141643736,A,G,LRP1B,-1,T,C,ATA,ACA,I1312T,T3935C,ATA>ACA,Missense,ENSP00000374135
TCGA-3M-AB46,2,179517425,C,T,TTN,-1,G,A,AGT,AAT,V13000M,G38998A,GTG>ATG,Missense,ENSP00000467141
TCGA-3M-AB46,2,179586588,T,A,TTN,-1,A,T,CAG,CTG,Q7601L,A22802T,CAG>CTG,Missense,ENSP00000467141
TCGA-3M-AB47,1,152284010,T,G,FLG,-1,A,C,CAT,CCT,I1118L,A3352C,ATC>CTC,Missense,ENSP00000357789
